In [1]:
import tensorflow as tf
import numpy as np
import pickle

with open('dictionary.pkl', 'rb') as fr:
    [char2id, id2char] = pickle.load(fr)

In [2]:
tf.reset_default_graph()
batch_size = 1
hidden_size = 256
num_layer = 2
embedding_size =256

X = tf.placeholder(tf.int32, [batch_size, None])
Y = tf.placeholder(tf.int32, [batch_size, None])
learning_rate = tf.Variable(0.0, trainable=False)

cell = tf.nn.rnn_cell.MultiRNNCell(
    [tf.nn.rnn_cell.LSTMCell(hidden_size, state_is_tuple=True) for i in range(num_layer)], 
    state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

embeddings = tf.Variable(tf.random_uniform([len(char2id) + 1, embedding_size], -1.0, 1.0))
embedded = tf.nn.embedding_lookup(embeddings, X)

outputs, last_states = tf.nn.dynamic_rnn(cell, embedded, initial_state=initial_state)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = tf.layers.dense(outputs, units=len(char2id) + 1)
probs = tf.nn.softmax(logits)
targets = tf.reshape(Y, [-1])

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))
params = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
#saver.restore(sess, tf.train.latest_checkpoint('./20'))





In [3]:
saver.restore(sess, tf.train.latest_checkpoint('./50'))
def generate():
    states_ = sess.run(initial_state)
    
    gen = ''
    c = '['
    while c != ']':
        gen += c
        x = np.zeros((batch_size, 1))
        x[:, 0] = char2id[c]
        probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
        probs_ = np.squeeze(probs_)
        pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
        c = id2char[pos]
    
    return gen[1:]

def generate_with_head(head):
    states_ = sess.run(initial_state)
    
    gen = ''
    c = '['
    i = 0
    while c != ']':
        gen += c
        x = np.zeros((batch_size, 1))
        x[:, 0] = char2id[c]
        probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
        probs_ = np.squeeze(probs_)
        pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))

        if (c == '[' or c == '。' or c == '，') and i < len(head):
            c = head[i]
            i += 1
        else:
            c = id2char[pos]
    
    return gen[1:]

print(generate())
print(generate_with_head('深度学习'))


INFO:tensorflow:Restoring parameters from ./50\50_epoch
许翁衰老爱街斜，三百年长已到家。不作邻翁坐炊饭，老农休说饭寒茶。
深宫虽在彩罗袍，度地遗踪竟早闻。学出何年权上遍，习孙须载黍诗豪。


In [4]:
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
print(generate())
#print(generate_with_head('天下无双'))

农涂种秫复相逢，滞货馋涎蚁也同。多下相思诗一句，何尝精积隔隣中。
流水阳沟细雨生，隋堤满渚早莺声。䌽芳门户催多少，睡里烧花欲遍江。
燕坐酣眠络乳力，萧萧疎雨点筠根。搘筇脚下何为寄，笑杀沙头自去门。
兀兀强临脩濑去，夜深无语立幽閒。悬知一叶非心瘦，野季重来似卧闲。
行遍洛川居士身，西南最值雨中鳞。黄金典合台前种，牵客老人每不勤。
捻在骊龙鹤，犹为秉炬愁。于今平地作，万壑也如秋。
山果多时阁，一冬梅半开。故教千一滓，曲径翠自开。
我来且小赋归欤，人如无端只一双。向晚须经头畔线，古今相厌转难存。
一溪烟雾四重波，半日祥烟入画梁。忧国一声乐报夜，一轮新火淑心时。
修春更有春宜雨，酒袖鸣牙叫不寒。寂历香林藏竹坞，谗披高蹈日头寒。
堕糁霜翎雨色疏，山风扫落叶声初。从来一叶扫除尽，四壁松烟挂碗如。
禁苑高花上玉阶，祥飚毬地昼萧回。玉人笑折金枝探，莫作清香宴殿来。
